# 구현할 코드 구조

![단층퍼셉트론](./image/single_layer_perceptron_CodeTree.PNG)

---

## 0.0 파이썬 모듈 불러오기

In [1]:
import csv
import numpy as np
# import time

np.random.seed(1234)

## 0.1 하이퍼 파라미터 정의

In [2]:
RND_MEAN = 0
RND_STD = 0.0030

LEARNING_RATE = 0.001 

## A.1 abalone_Exec
- 실험용 메인함수

In [9]:
def abalone_Exec(epoch_count=10, mb_size=10, report=1):
    
    load_abalone_dataset() # 데이터 로드
    
    init_model() # 모델 초기화
    
    train_and_test(epoch_count, mb_size, report) # 학습 및 테스트 

## B.1 load_abalone_dataset
- 데이터 적재함수 정의
    
    1. 데이터 불러오기
    2. 전역 변수 생성
    3. 원-핫 인코딩 처리

In [22]:
def load_abalone_dataset():
    with open('./data/abalone.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None) # 데이터의 첫번째 행(칼럼명)을 생략
        rows = []
        for row in csvreader:
            rows.append(row)
            
    global data, input_cnt, output_cnt 
    input_cnt, output_cnt = 10, 1 # 입,출력 벡터 크기
    data = np.zeros([len(rows), input_cnt+output_cnt])

    for n, row in enumerate(rows):
        if row[0] == 'I': data[n, 0] = 1 # if문 짧게 쓰는 방법
        if row[0] == 'M': data[n, 1] = 1
        if row[0] == 'F': data[n, 2] = 1
        data[n, 3:] = row[1:]

---
- next() 활용 예제

In [7]:
# next() 사용 O
with open('./data/abalone.csv') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader, None) 
    rows = []
    for row in csvreader:
        rows.append(row)
print(rows[0])
print(len(rows))

['M', '0.455', '0.365', '0.095', '0.514', '0.2245', '0.101', '0.15', '15']
4177


In [8]:
# next() 사용 X
with open('./data/abalone.csv') as csvfile:
    csvreader = csv.reader(csvfile)
#     next(csvreader, None) 
    rows = []
    for row in csvreader:
        rows.append(row)
print(rows[0])
print(len(rows))

['Sex', 'Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight', 'Rings']
4178


---

## B.2 init_model
- 파라미터 (랜덤) 초기화 함수

In [16]:
def init_model():
    global weight, bias, input_cnt, output_cnt
    weight = np.random.normal(RND_MEAN, RND_STD, [input_cnt, output_cnt])
                            # 평균,표준편차,크기
    bias = np.zeros([output_cnt]) # bias는 초반에 영향을 미칠 수 있으므로 처음엔 0으로 지정

## B.3 train_and_test
- 학습 및 평가 함수 정의
    - 하위 함수가 여러 개 있음. 주의.
    
    
    1. 학습
    2. 출력

In [ ]:
def train_and_test(epoch_count, mb_size, report):
                   # mb_size : 미니배치크기
                   # report  : 출력 주기
            
    step_count = arrange_data(mb_size) # 미니배치크기에 따른 step_count
    test_x, test_y = get_test_data()
    
    for epoch in range(epoch_count):
        losses, accs = [], [] # 손실함수, 정확도 
        
        for n in range(step_count):
            train_x, train_y = get_train_data(mb_size, n)
            loss, acc = run_train(train_x, train_y)
            losses.append(loss) # step_count 개수만큼의 원소가 생성됨
            accs.append(acc)    # step_count 개수만큼의 원소가 생성됨
                                # loss, accs는 이 후에 평균낼 것
            
        if report > 0 and (epoch+1) % report == 0: # 출력 주기 조건
            # report > 10이라 하면 10개 이후부터 출력하겠다는 의미
            acc = run_test(test_x, test_y)
            print('Epoch {}: loss={:5.3f}, accuracy={:5.3f}/{:5.3f}'. \
                  format(epoch+1, np.mean(losses), np.mean(accs), acc))
                              # np.mean(accs) : train accracy
                              # acc           : test accuracy
            
    final_acc = run_test(test_x, test_y)
    print('\nFinal Test: final accuracy = {:5.3f}'.format(final_acc))

---

- arrange_data()의 반환값, 미니배치 스텝 수란

In [26]:
print("총 데이터의 수(행)", data.shape[0])

load_abalone_dataset()

mb_size = 100
step_count = int(data.shape[0] * 0.8) // mb_size  # train data는 전체의 80%
                                                  # 100개씩 데이터를 처리하면 33step이 걸림
print("데이터의 80%의 미니배치 스텝수 :", step_count)
print(step_count * mb_size)

총 데이터의 수(행) 4177
데이터의 80%의 미니배치 스텝수 : 33
3300


---